# Setelan Awal Notebook
Berikut adalah langkah-langkah untuk memulai instalasi pustaka yang diperlukan dan mengunduh model:

In [1]:
!pip install -Uqqq pip --progress-bar off
# Mengupdate pip ke versi terbaru tanpa menampilkan progress bar

!pip install -qqq torch==2.0.1 --progress-bar off
# Menginstal pustaka PyTorch versi 2.0.1 tanpa menampilkan progress bar

!pip install -qqq transformers==4.31.0 --progress-bar off
# Menginstal pustaka Transformers versi 4.31.0 tanpa menampilkan progress bar

!pip install -qqq langchain==0.0.266 --progress-bar off
# Menginstal pustaka LangChain versi 0.0.266 tanpa menampilkan progress bar

!pip install -qqq chromadb==0.4.5 --progress-bar off
# Menginstal pustaka ChromaDB versi 0.4.5 tanpa menampilkan progress bar

!pip install -qqq pypdf==3.15.0 --progress-bar off
# Menginstal pustaka PyPDF versi 3.15.0 tanpa menampilkan progress bar

!pip install -qqq xformers==0.0.20 --progress-bar off
# Menginstal pustaka XFormers versi 0.0.20 tanpa menampilkan progress bar

!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
# Menginstal pustaka SentenceTransformers versi 2.2.2 tanpa menampilkan progress bar

!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
# Menginstal pustaka InstructorEmbedding versi 1.0.1 tanpa menampilkan progress bar

!pip install -qqq pdf2image==1.16.3 --progress-bar off
# Menginstal pustaka PDF2Image versi 1.16.3 tanpa menampilkan progress bar

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.3 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires pydantic>=2, but you have pydantic 1.10.15 which is incompatible.


Untuk AutoGPTQ, librarinya khusus karena harus sama dengan CUDA biar bisa pakai GPU. AutoGPTQ bukan automl yaa, itu tuh ,ibrary biar model yang gede jadi lebih efisien pengolahannya, caranya itu ya dengan kuantisasi weight.

In [2]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.0/auto_gptq-0.4.0+cu118-cp310-cp310-linux_x86_64.whl
!pip install -qqq auto_gptq-0.4.0+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

Nah biar nanti kita bisa review pdf yang kita masukin, bisa pakai poppler-utils. Fungsinya untuk ubah pdf jadi gambar.

In [3]:
!sudo apt-get install poppler-utils -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 65 not upgraded.
Need to get 2564 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.4 [916 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.4 [174 kB]
Fetched 2564 kB in 4s (571 kB/s)       
Selecting previously unselected package 

Setelah semuanya terinstall, kita import library yang dibutuhkan aja.

In [4]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline
 
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

ModuleNotFoundError: No module named 'pydantic_v1.class_validators'; 'pydantic_v1' is not a package

# Data

In [ ]:
!mkdir pdfs
!gdown 1v-Rn1FVU1pLTAQEgm0N9oB6cExMoebZr -O pdfs/tesla-earnings-report.pdf
!gdown 1Xc890jrQvCExAkryVWAttsv1DBLdVefN -O pdfs/nvidia-earnings-report.pdf
!gdown 1Epz-SQ3idPpoz75GlTzzomag8gplzLv8 -O pdfs/meta-earnings-report.pdf

# Load Documents

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
len(docs)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

In [ ]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

In [ ]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"
 
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
 
model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device="cuda:0",
    quantize_config=None,
)

# Prompt Format

In [ ]:
[INST] <<SYS>> {system_prompt} <</SYS>>
 
{prompt} [/INST]

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully
as possible, while being safe. Your answers should not include any harmful,
unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your
responses are socially unbiased and positive in nature.
 
If a question does not make any sense, or is not factually coherent, explain
why instead of answering something not correct. If you don't know the answer to a
question, please don't share false information.
""".strip()
 
 
def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>
 
{prompt} [/INST]
""".strip()

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
 
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

# Chain

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."
 
template = generate_prompt(
    """
{context}
 
Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)
 
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

# Chat PDF

In [ ]:
result = qa_chain("What is the per share revenue for Meta during 2023?")

In [ ]:
len(result["source_documents"])

In [ ]:
print(result["source_documents"][0].page_content)

In [ ]:
result = qa_chain("What is the per share revenue for Tesla during 2023?")

In [ ]:
result = qa_chain("What is the per share revenue for Nvidia during 2023?")

In [ ]:
result = qa_chain("What is the estimated YOY revenue for Meta during 2023?")

In [ ]:
result = qa_chain("Choose one company to invest (Tesla, Nvidia or Meta) to maximize your profits for the long term (10+ years)?")